Weather data preparation 

In [0]:
import pandas as pd
import numpy as np

In [0]:
#Works on a single county basis
county_name = 'ALLEGANY'

In [284]:
#Read master csv file considering first column in the dataset as the index column
df_master = pd.read_csv("master.csv", index_col=0)

#df_master = df_master.drop(['County Centroid', 'Disease_INFLUENZA_A', 'Disease_INFLUENZA_B', 'Disease_INFLUENZA_UNSPECIFIED', 'fluSymptoms', 'soreThroat', 'VaxRate', 'Population', 'GDP_change'], axis=1)   #Serves no purpose - drop the columns just for accomodating all the required columns in the same sheet (easy for debugging)
print(df_master['County'].unique())
df_master.head()

['ALBANY' 'ALLEGANY' 'BRONX' 'BROOME' 'CATTARAUGUS' 'CAYUGA' 'CHAUTAUQUA'
 'CHEMUNG' 'CHENANGO' 'CLINTON' 'COLUMBIA' 'CORTLAND' 'DELAWARE'
 'DUTCHESS' 'ERIE' 'ESSEX' 'FRANKLIN' 'FULTON' 'GENESEE' 'GREENE'
 'HAMILTON' 'HERKIMER' 'JEFFERSON' 'KINGS' 'LEWIS' 'LIVINGSTON' 'MADISON'
 'MONROE' 'MONTGOMERY' 'NASSAU' 'NEW YORK' 'NIAGARA' 'ONEIDA' 'ONONDAGA'
 'ONTARIO' 'ORANGE' 'ORLEANS' 'OSWEGO' 'OTSEGO' 'PUTNAM' 'QUEENS'
 'RENSSELAER' 'RICHMOND' 'ROCKLAND' 'SARATOGA' 'SCHENECTADY' 'SCHOHARIE'
 'SCHUYLER' 'SENECA' 'ST LAWRENCE' 'STEUBEN' 'SUFFOLK' 'SULLIVAN' 'TIOGA'
 'TOMPKINS' 'ULSTER' 'WARREN' 'WASHINGTON' 'WAYNE' 'WESTCHESTER' 'WYOMING'
 'YATES']


,Unnamed: 0.1,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change,Population,flu,fluSymptoms,fever,cough,soreThroat,Target,VaxRate
0,0,ALBANY,36001,"(42.5882713, -73.9740136)",2009,40,10/10/2009,4,4,0,0,6.8,-,-,-,0.211869,0.174420,0.206388,0.112639,0.087600,27.0,NaN
1,1,ALBANY,36001,"(42.5882713, -73.9740136)",2009,41,10/17/2009,11,11,0,0,6.8,-,-,-,0.334176,0.176985,0.138915,0.083486,0.067525,87.0,NaN
2,2,ALBANY,36001,"(42.5882713, -73.9740136)",2009,42,10/24/2009,27,27,0,0,6.8,-,-,-,0.761100,0.657923,0.492075,0.156241,0.091104,77.0,NaN
3,3,ALBANY,36001,"(42.5882713, -73.9740136)",2009,43,10/31/2009,87,86,1,0,6.8,-,-,-,0.849600,0.855000,0.575424,0.214376,0.078840,61.0,NaN
4,4,ALBANY,36001,"(42.5882713, -73.9740136)",2009,44,11/07/2009,77,76,1,0,6.8,-,-,-,0.530469,0.738720,0.396900,0.246318,0.125195,29.0,NaN


In [291]:
#Read CSV file

df_TPS_1 = pd.read_csv(county_name.title() + ".csv")
df_TPS_1.head()
df_TPS_1 = df_TPS_1.rename(columns={" Date": "DATE", " Precipitation": "PRCP", " MaxTemperature": "TMAX", " MinTemperature": "TMIN", " AvgTemperature": "TAVG", " Snowfall": "SNOW", " SnowDepth": "SNOWDepth"})
#print(df_TPS_1.head())

df_TPS_1['PRCP'] = pd.to_numeric(df_TPS_1['PRCP'], errors='coerce')
df_TPS_1['SNOW'] = pd.to_numeric(df_TPS_1['SNOW'], errors='coerce')
df_TPS_1['TMAX'] = pd.to_numeric(df_TPS_1['TMAX'], errors='coerce')
df_TPS_1['TMIN'] = pd.to_numeric(df_TPS_1['TMIN'], errors='coerce')
df_TPS_1['TAVG'] = pd.to_numeric(df_TPS_1['TAVG'], errors='coerce')
df_TPS_1['SNOWDepth'] = pd.to_numeric(df_TPS_1['SNOWDepth'], errors='coerce')
#print(df_TPS_1.head())
#print(type(df_TPS_1['PRCP'][1016]))
#print(type(df_TPS_1['SNOW'][1016]))


#print(df_TPS_1.applymap(np.isreal))
df_TPS_1.sample(10)
#df_TPS_1.to_csv("temp.csv")

,Date,TMAX,TMIN,TAVG,PRCP,SNOW,SNOWDepth
3368,2018-11-21,34.0,26.0,30.0,0.04,0.5,7.0
1712,2014-05-10,84.0,47.0,65.5,0.23,0.0,0.0
1529,2013-11-08,47.0,30.0,38.5,0.10,NaN,NaN
1168,2012-11-12,69.0,41.0,55.0,0.00,NaN,NaN
320,2010-07-18,87.0,56.0,71.5,0.40,0.0,0.0
1792,2014-07-29,67.0,49.0,58.0,0.20,0.0,0.0
1438,2013-08-09,83.0,63.0,73.0,0.42,0.0,0.0
2870,2017-07-11,75.0,52.0,63.5,0.05,0.0,0.0
2239,2015-10-19,38.0,26.0,32.0,0.17,NaN,0.0
355,2010-08-22,77.0,54.0,65.5,1.24,0.0,0.0


In [292]:
#Convert df_TPS['DATE'] to pandas Date format
df_TPS_1['Date'] = pd.to_datetime(df_TPS_1['Date'])
#Extract Week number from date
df_TPS_1['Week_Number'] = df_TPS_1['Date'].dt.week
#Extract Year number from date
df_TPS_1['Year'] = df_TPS_1['Date'].dt.year
df_TPS_1.fillna(df_TPS_1.groupby(df_TPS_1.Date.dt.month).transform('mean'))
#df_TPS_1.to_csv("temp2.csv")

#df_TPS_1 = df_TPS_1.drop(['Date'], axis = 1)
print(df_TPS_1.sample(10))

#Create a dict to denote how a feature has to be grouped 
#Eg: PRCP : mean - PRCP features are averaged over complete week
agg_dict = {'PRCP'  : 'mean',
            'SNOW'  : 'mean',
            'TAVG'  : 'mean', 
            'TMAX'  : 'mean',
            'TMIN'  : 'mean',
            'SNOWDepth' : 'mean'}

#Group same Year and WeekNumnber as per the values in the agg_dict
df_TPS_1 = df_TPS_1.groupby(['Year','Week_Number']).agg(agg_dict)
#Year and Week_NUmber are in index format, convert them into columns
df_TPS_1.reset_index(inplace=True)
#Take mean of TMAX and TMIN to get TAVG (TMAX and TMIN here are already averages of the entire week)
#df_TPS['TAVG'] = (df_TPS['TMAX'] + df_TPS['TMIN'])/2
df_TPS_1.sample(10)

           Date  TMAX  TMIN  TAVG  PRCP  SNOW  SNOWDepth  Week_Number  Year
472  2010-12-17  23.0   9.0  16.0   NaN   0.1        2.0           50  2010
3341 2018-10-25  47.0  31.0  39.0  0.02   0.0        0.0           43  2018
1309 2013-04-02  41.0  19.0  30.0  0.20   2.0        2.0           14  2013
1323 2013-04-16  69.0  39.0  54.0  0.00   0.0        0.0           16  2013
1733 2014-05-31  73.0  46.0  59.5  0.00   0.0        0.0           22  2014
3565 2019-06-06  74.0  62.0  68.0  0.81   0.0        0.0           23  2019
1057 2012-07-24  88.0  66.0  77.0  0.15   NaN        NaN           30  2012
591  2011-04-15  60.0  29.0  44.5  0.00   0.0        0.0           15  2011
511  2011-01-25  23.0 -19.0   2.0  0.00   0.0        6.0            4  2011
2425 2016-04-22  72.0  31.0  51.5  0.02   0.0        0.0           16  2016


,Year,Week_Number,PRCP,SNOW,TAVG,TMAX,TMIN,SNOWDepth
457,2018,21,0.151429,0.000000,61.071429,73.285714,48.857143,0.000000
150,2012,28,0.000000,NaN,68.571429,84.714286,52.428571,NaN
474,2018,38,0.237143,0.000000,65.857143,74.428571,57.285714,0.000000
392,2017,8,0.082857,0.028571,45.500000,58.500000,31.428571,0.000000
488,2018,52,0.000000,0.000000,32.428571,36.285714,28.571429,0.000000
179,2013,5,0.378000,0.600000,25.500000,36.142857,14.857143,1.142857
56,2010,39,0.470000,0.000000,53.571429,60.857143,46.285714,0.000000
460,2018,24,0.122857,0.000000,64.000000,76.428571,51.571429,0.000000
389,2017,5,0.031667,0.383333,21.428571,27.857143,15.000000,1.500000
250,2014,24,0.300000,0.000000,64.214286,73.428571,55.000000,0.000000


In [0]:
df_TPS_1.sample(10)

,Year,Week_Number,PRCP,SNOW,TAVG,TMAX,TMIN,SNOWDepth
111,2011,49,0.278497,0.165223,36.071429,43.571429,28.571429,0.000000
38,2010,29,0.286185,0.000000,73.928571,82.714286,65.142857,0.000000
302,2015,32,0.095313,0.000000,71.714286,82.857143,60.571429,0.000000
66,2011,4,0.078489,0.414835,19.000000,26.285714,11.714286,12.428571
13,2010,4,0.224203,0.243407,23.571429,31.857143,15.285714,1.189985
414,2017,38,0.000000,0.000000,70.857143,82.714286,59.000000,0.000000
53,2010,44,0.164286,0.000000,39.857143,46.857143,32.857143,0.000000
370,2016,47,0.050182,0.044382,35.428571,40.000000,30.857143,0.000000
417,2017,41,0.094286,0.000000,60.500000,69.857143,51.142857,0.000000
336,2016,13,0.101533,0.300000,45.857143,55.428571,36.285714,0.285714


In [0]:
#Creates a new dataframe by merging only the required county in master dataframe and the df_TPS dataframe for that county
#This new dataframe is only a partial dataframe that contains only the rows that have the mentioned county values
#Complete dataset has to be created by either merging or creating df_TPS for all counties and cerging all temp_df
temp_df = pd.merge(df_master.loc[df_master['County'] == county_name], df_TPS_1,  how='left', left_on=['year','CDC Week'], right_on = ['Year','Week_Number'])

In [294]:
#Drop the newly created Year and Week_Number columns as our master dataset already has these columns
temp_df = temp_df.drop(['Year', 'Week_Number'], axis = 1)
temp_df.sample(5)
temp_df.shape

(302, 28)

In [0]:
master_updated = pd.concat([master_updated, temp_df])

In [296]:
master_updated.shape

(604, 28)

In [298]:
master_updated.sample(10)
#master_updated.to_csv("Albany_Allegany.csv")

,Unnamed: 0.1,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change,Population,flu,fluSymptoms,fever,cough,soreThroat,Target,VaxRate,PRCP,SNOW,TAVG,TMAX,TMIN,SNOWDepth
176,478,ALLEGANY,36003,"(42.2478938, -78.0261758)",2015,51,12/26/2015,0,0,0,0,6.0,1072033.0,-2.6,47324.0,0.008368,0.005387,0.274031,0.586236,0.512047,0.0,0.534,0.097143,0.142857,40.785714,46.571429,35.000000,0.142857
179,481,ALLEGANY,36003,"(42.2478938, -78.0261758)",2016,3,01/23/2016,0,0,0,0,6.3,-,-,47039.0,0.005313,0.002694,0.247246,0.463180,0.428487,0.0,0.575,0.040000,0.600000,14.500000,21.000000,8.000000,1.400000
50,50,ALBANY,36001,"(42.5882713, -73.9740136)",2011,6,02/12/2011,3,3,0,0,6.9,-,-,304564.0,0.004779,0.002821,0.309825,0.127764,0.229877,5.0,NaN,0.052500,0.500000,20.285714,30.857143,9.714286,15.142857
299,299,ALBANY,36001,"(42.5882713, -73.9740136)",2019,42,10/19/2019,0,0,0,0,-,-,-,-,NaN,NaN,NaN,NaN,NaN,2.0,NaN,0.467143,0.000000,48.928571,59.000000,38.857143,0.000000
91,393,ALLEGANY,36003,"(42.2478938, -78.0261758)",2012,13,03/31/2012,0,0,0,0,8.4,1016724.0,-,48201.0,0.001472,0.000000,0.185456,0.131040,0.224721,0.0,NaN,0.020000,0.000000,38.571429,48.285714,28.857143,0.000000
215,517,ALLEGANY,36003,"(42.2478938, -78.0261758)",2017,5,02/04/2017,15,15,0,0,6.6,-,-,46688.0,0.025103,0.017811,0.532266,0.650838,0.603623,41.0,0.591,0.031667,0.383333,21.428571,27.857143,15.000000,1.500000
300,300,ALBANY,36001,"(42.5882713, -73.9740136)",2019,43,10/26/2019,2,0,0,2,-,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.332500,0.000000,51.357143,60.142857,42.571429,0.000000
168,168,ALBANY,36001,"(42.5882713, -73.9740136)",2015,43,10/31/2015,0,0,0,0,4.0,26254937.0,-0.6,307476.0,0.013415,0.007171,0.166141,0.224057,0.274645,0.0,0.362,0.078571,0.000000,47.000000,56.857143,37.142857,0.000000
231,533,ALLEGANY,36003,"(42.2478938, -78.0261758)",2017,40,10/07/2017,0,0,0,0,6.6,-,-,46688.0,0.016934,0.001979,0.256861,0.364469,0.326466,0.0,0.337,0.045714,0.000000,56.785714,71.571429,42.000000,0.000000
119,119,ALBANY,36001,"(42.5882713, -73.9740136)",2013,8,02/23/2013,13,3,9,1,6.1,25479702.0,2.8,306613.0,0.003982,0.009405,0.333558,0.356675,0.286306,10.0,NaN,0.035000,0.150000,27.857143,34.285714,21.428571,0.000000
